In [1]:
%run titolari_e_panchinari.ipynb

In [2]:
create = False

ultima_giornata = 38

In [3]:
l = range(6, 20)
excel = "Quotazioni_Fantacalcio_Stagione_2022_23.xlsx"
if create:
    [modello_fantacalcio(ultima_giornata, n, save=create, file_quotazioni=excel) for n in l]

In [6]:
def read_final_result(ultima_giornata, n_giornate, lega, team='Io'):
    listone = pd.read_excel("Listone_produzione.xlsx")
    team = listone[(listone.Lega == lega) & (listone.Proprietario == team)].Nome.tolist()
    lista_nomi = '("'+'", "'.join(team) + '")'
    return pd.read_excel(f"final_result/giornata_{ultima_giornata}/final_result_ultime_{n_giornate}.xlsx").query(f'Nome in {lista_nomi}')

In [7]:
dfs = [read_final_result(ultima_giornata, n, "FANTABERTEBOOM")[["R", "Nome", "Squadra", "Media", "FantaMedia", "Voto1", "FantaVoto1", "Posizione"]].rename(columns={"Voto1": "Voto", "FantaVoto1": "FantaVoto", "Media": "VotoPotenziale","FantaMedia": "FantaVotoPotenziale" }) for n in l]
dfs_avversario = [read_final_result(ultima_giornata, n, "FANTABERTEBOOM", "Erik")[["R", "Nome", "Squadra", "Media", "FantaMedia", "Voto1", "FantaVoto1", "Posizione"]].rename(columns={"Voto1": "Voto", "FantaVoto1": "FantaVoto", "Media": "VotoPotenziale","FantaMedia": "FantaVotoPotenziale" }) for n in l]

In [13]:
dfs_avversario[0]
# dfs[0]

,R,Nome,Squadra,VotoPotenziale,FantaVotoPotenziale,Voto,FantaVoto,Posizione
0,P,SZCZESNY,Juventus,6.500000,6.200000,6.50,6.00,1
2,P,DRAGOWSKI,Spezia,6.500000,5.750000,6.50,6.00,3
22,D,GOSENS,Inter,6.166667,7.166667,6.00,7.00,7
27,D,TOLOI,Atalanta,6.125000,6.625000,6.00,6.50,12
60,D,UDOGIE,Udinese,5.800000,5.800000,6.00,6.00,45
73,D,VOJVODA,Torino,5.666667,5.666667,5.50,5.50,58
108,C,MILINKOVIC-SAVIC,Lazio,6.333333,7.416667,6.50,7.50,8
111,C,FELIPE ANDERSON,Lazio,6.250000,7.333333,6.00,7.00,11
116,C,ZIELINSKI,Napoli,6.166667,7.000000,6.00,7.00,16
153,C,LUKIC,Torino,6.000000,5.916667,6.00,6.00,53


In [14]:
from pandasql import sqldf
import pandas as pd
from sklearn import datasets
total = pd.concat(dfs)
total_avversario = pd.concat(dfs_avversario)

In [15]:
def avg_df(tot_df):
    view_name = f'{tot_df=}'.split('=')[0]
    query = f"""select R, Nome, Squadra, VotoPotenziale, FantaVotoPotenziale, Voto, FantaVoto
                from (
                    select 
                        R,
                        Nome,
                        Squadra,
                        AVG(VotoPotenziale) as VotoPotenziale,
                        AVG(FantaVotoPotenziale) as FantaVotoPotenziale,
                        AVG(Voto) as Voto,
                        AVG(FantaVoto) as FantaVoto
                    from {view_name}
                    group by R, Nome, Squadra
                    having count(*) >= 14
                )
                order by R desc, FantaVoto desc, Voto desc"""
    return sqldf(query)

In [16]:
tot = avg_df(total)
tot_avversario = avg_df(total_avversario)

In [17]:
other = ["NUYTINCK"]
squadra_titolare, panchinari, listone = titolari_e_panchinari(dfs, other=other)
squadra_titolare

4-3-3 78.46428571428571


,R,Nome,Squadra,counts
12,P,HANDANOVIC,Inter,14
8,D,BREMER,Juventus,14
9,D,CALABRIA,Milan,14
10,D,RRAHMANI,Napoli,14
11,D,SINGO,Torino,14
4,C,BARAK,Verona,14
5,C,CANDREVA,Sampdoria,14
2,C,MKHITARYAN,Inter,13
0,A,BERARDI,Sassuolo,14
3,A,RAFAEL LEAO,Milan,14


In [19]:
panchinari

,R,Nome,VotoPotenziale,FantaVotoPotenziale
7,A,QUAGLIARELLA,6.022185,6.796776
9,A,SANABRIA,6.073625,6.686224
10,A,BONAZZOLI,6.259152,6.619605
12,C,ZURKOWSKI,6.122325,6.260398
14,C,EL SHAARAWY,5.768900,5.922925
15,D,DE SILVESTRI,5.831824,5.870546
16,D,MARUSIC,5.531575,5.566717
17,D,STOJANOVIC,5.686748,5.541615
19,P,SKORUPSKI,6.105427,3.889185


In [33]:
other = [""]
squadra_titolare_avversario, panchinari_avversario, listone_avversario = titolari_e_panchinari(dfs_avversario, other=other)
squadra_titolare_avversario

4-3-3 78.28571428571429


,R,Nome,Squadra,counts
7,P,SZCZESNY,Juventus,13
9,D,GOSENS,Inter,14
12,D,UDOGIE,Udinese,14
13,D,VOJVODA,Torino,14
5,D,TOLOI,Atalanta,10
4,C,FELIPE ANDERSON,Lazio,14
6,C,MILINKOVIC-SAVIC,Lazio,14
3,C,ZIELINSKI,Napoli,5
1,A,OSIMHEN,Napoli,14
0,A,SIMEONE,Verona,11


In [45]:
panchinari

,R,Nome,VotoPotenziale,FantaVotoPotenziale
7,A,QUAGLIARELLA,6.022185,6.796776
9,A,SANABRIA,6.073625,6.686224
10,A,BONAZZOLI,6.259152,6.619605
12,C,ZURKOWSKI,6.122325,6.260398
14,C,EL SHAARAWY,5.768900,5.922925
15,D,DE SILVESTRI,5.831824,5.870546
16,D,MARUSIC,5.531575,5.566717
17,D,STOJANOVIC,5.686748,5.541615
19,P,SKORUPSKI,6.105427,3.889185


In [ ]:
giocatori = ["Dragowski", "Fuzato", "Rui Patricio", "Alex Sandro", "Criscito", "De Ligt", "Hateboer", "Hysaj", "Maehle", "Maksimovic", "Muldur", "Bernardeschi", "Candreva", "Felipe Anderson", "Freuler", "Perisic", "Pessina", "Saponara", "Soriano", "Brekalo", "Kean", "Martinez L.", "Mertens", "Osimhen", "Piatek"]
lista_giocatori_titolari = [x.upper() for x in giocatori]
squadra_titolare = titolari_e_panchinari(dfs, 5, lista_giocatori_titolari = lista_giocatori_titolari)
squadra_titolare

In [ ]:
modello_fantacalcio(38, 38, True, 0.5, file_quotazioni="Quotazioni_Fantacalcio_Stagione_2022_23.xlsx")